In [1]:
import numpy as np
import scipy.stats
import scs

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    z = np.random.randn(m)
    y = pos(z)
    s = y - z

    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
    U, S, V = np.linalg.svd(A, full_matrices=False)
    S = S**2
    S /= np.max(S)
    A = (U * S) @ V
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = A.dot(x) + s
    
    b /= np.linalg.norm(b)
    c /= np.linalg.norm(c)

    return (P, A, b, c)

In [3]:
def is_optimal(P, A, b, c, x, y, tol=1e-6):
    s = b - A @ x
    
    if (np.linalg.norm(P @ x + c + A.T @ y) < tol and
        np.abs(y.T @ s) < tol and
        np.linalg.norm(s - pos(s)) < tol and
        np.linalg.norm(y - pos(y)) < tol):
        return True
    return False

In [4]:
# ''linear'' projection logic

class LinearProjector(object):
    def __init__(self, P, A, b, c):
        (m,n) = A.shape
        self.A = A
        self.h = np.hstack((c, b))        
        self.L = scipy.linalg.cho_factor(np.eye(n) + P + A.T @ A)
        self.g = self._solve(self.h)

    def _solve(self, v):
        (m, n) = self.A.shape
        sol = np.zeros(n+m,)
        sol[:n] = scipy.linalg.cho_solve(self.L, v[:n] - self.A.T @ v[n:])
        sol[n:] = v[n:] + self.A @ sol[:n]
        return sol
        
    def project(self, w):
        g = self.g
        p = self._solve(w[:-1])
        
        _a = 1 + g.T@g
        _b = w[:-1].T @ g - 2 * p.T @ g - w[-1]
        _c = p.T@p - w[:-1].T @ p
        
        tau = (-_b + np.sqrt(_b ** 2 - 4 * _a * _c)) / 2 / _a
        
        return np.hstack((p - tau * g, tau))

# Douglas-Rachford splitting / ADMM for QP
def solve_qp_dr(P, A, b, c, N=1000):
    (m,n) = np.shape(A)

    lp = LinearProjector(P, A, b, c)

    u = np.zeros(n+m+1,)
    #u[:n+m] = np.hstack((c, b))
    u[-1] = 1.

    def proj_cone(v):
        v[n:] = np.maximum(v[n:], 0.)
        return v


    use_dr = True # slightly different DR vs ADMM
    
    for i in range(N):
        # for DR ut converges to sol *not* u, see Patrinos, Stella, Bemporad, 2014
        # v - ut go to zero
        dr_lam = 0.5 # \in [0,1], 0.5 = DR, 1.0 = PR

        ut = lp.project(u)
        v = proj_cone(2 * ut - u)
        u += 2 * dr_lam * (v - ut)
        
        x = ut[:n] / ut[-1]
        y = ut[n:-1] / ut[-1]
        
        if is_optimal(P, A, b, c, x, y, tol=1e-9):
            break
        
    print(i)
    return x,y



In [5]:
m = 15
n = 10
N = int(5e3)

In [6]:
feas_h_is = []
feas_nh_is = []

conds = []

np.random.seed(123)
(P, A, b, c) = gen_feasible(m, n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)
print(y)

514
[-0.13351969  0.09140453  0.11516882  0.28373837 -0.07264948 -0.2798121
 -0.25180523  0.30405864 -0.13665965 -0.25222408]
[ 3.54744623e-10  1.46516000e-01  3.08666010e-10  8.37947276e-12
  1.13901231e-10  2.93315538e-01 -6.08878503e-11 -1.31061643e-10
  2.08074169e-01 -1.85143373e-10  1.06664439e-10 -1.35809290e-10
 -4.09269196e-11 -4.27421504e-10 -1.45058328e-10]


In [7]:
np.random.seed(123)
(P, A, b, c) = gen_feasible(m, n)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)

cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 5, eps=1e-9)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 5, eps=1e-9, use_indirect=True)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 150, nnz in P = 55
eps = 1.00e-09, alpha = 1.50, max_iters = 5000, normalize = 0
acceleration_lookback = 5, rho_x = 1.00e-03
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 9.04e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.28e+01  4.32e+01  9.77e-01 -3.79e+01 -1.32e-02  0.00e+00  6.41e-03 
    60| 1.84e-14  1.11e-14  1.38e-15 -4.35e-01 -2.74e-02  0.00e+00  7.17e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 7.25e-03s
	Lin-sy

{'x': array([-0.1335197 ,  0.09140452,  0.11516882,  0.28373838, -0.07264949,
        -0.27981211, -0.25180523,  0.30405864, -0.13665965, -0.25222409]),
 'y': array([0.        , 0.14651601, 0.        , 0.        , 0.        ,
        0.29331554, 0.        , 0.        , 0.20807417, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 's': array([0.27191369, 0.        , 0.01001885, 0.4654744 , 0.17485928,
        0.        , 0.74165786, 0.11940615, 0.        , 0.25756337,
        0.21708059, 0.00810246, 0.00398306, 0.17112443, 0.18983148]),
 'info': {'statusVal': 1,
  'iter': 60,
  'pobj': -0.43480706664709184,
  'dobj': -0.027406651563679826,
  'resPri': 1.5065272131910306e-14,
  'resDual': 3.7933483407616406e-13,
  'relGap': 6.64805292388047e-14,
  'resInfeas': nan,
  'resUnbdd': 2.299870624714669,
  'solveTime': 8.839588,
  'setupTime': 6.438073,
  'status': 'Solved'}}

In [8]:
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 0, eps=1e-9, use_indirect=True, alpha=1.0, rho_x=1.0)

----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 150, nnz in P = 55, cg_rate = 2.00
eps = 1.00e-09, alpha = 1.00, max_iters = 5000, normalize = 0
acceleration_lookback = 0, rho_x = 1.00e+00
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 9.20e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.73e+19  1.24e+19  9.20e-01 -1.01e+19  4.92e+15  1.38e+18  9.28e-03 
   100| 9.08e-05  3.02e-04  1.21e-04 -4.35e-01 -2.76e-02  0.00e+00  1.03e-02 
   200| 1.48e-05  4.43e-06  2.38e-06 -4.35e-01 -2.74e-02  0.00e+00  1.14e-02 
   300| 1.12e-07  9.15e-07  3.90e

{'x': array([-0.1335197 ,  0.09140452,  0.11516883,  0.28373838, -0.0726495 ,
        -0.27981212, -0.25180523,  0.30405864, -0.13665965, -0.25222409]),
 'y': array([0.        , 0.146516  , 0.        , 0.        , 0.        ,
        0.29331555, 0.        , 0.        , 0.20807417, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]),
 's': array([0.27191369, 0.        , 0.01001884, 0.4654744 , 0.17485928,
        0.        , 0.74165786, 0.11940615, 0.        , 0.25756337,
        0.21708059, 0.00810246, 0.00398306, 0.17112443, 0.18983148]),
 'info': {'statusVal': 1,
  'iter': 700,
  'pobj': -0.43480706679843645,
  'dobj': -0.02740665102875111,
  'resPri': 4.4734787141664915e-10,
  'resDual': 5.724493483232214e-10,
  'relGap': 2.8188718644258197e-10,
  'resInfeas': nan,
  'resUnbdd': 2.2998706240232774,
  'solveTime': 18.112716,
  'setupTime': 9.19522,
  'status': 'Solved'}}

In [9]:
np.random.seed(123)
(P, A, b, c) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 10, eps=1e-9)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 10, eps=1e-9)

4999
[-0.09617189  0.63333043  0.29524907  0.20594927 -0.04627224 -0.09247665
 -0.05887478  0.38069075  0.31876822 -0.41737046]
----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 150, nnz in P = 0
eps = 1.00e-09, alpha = 1.50, max_iters = 5000, normalize = 0
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 6.50e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.31e+19  4.37e+18  9.01e-01 -6.13e+19 -3.21e+18  2.67e+19  7.29e-03 
   100| 2.62e-05  6.49e-06  2.05e-05 -2.18e-01 -2.18e-01  0.00e+00  8.53e-03 
 

{'x': array([ 0.26346588,  0.95789739, -0.04044451, -0.2960109 , -0.64612837,
        -1.18772395, -1.70178189,  1.62430736, -1.10823861, -1.9453558 ]),
 'y': array([0.        , 0.94962607, 0.269439  , 0.        , 0.        ,
        1.57242126, 0.        , 0.        , 1.20536578, 0.        ,
        0.        , 0.        , 1.42003202, 0.        , 0.        ]),
 's': array([0.58688793, 0.        , 0.        , 0.29871156, 0.31012377,
        0.        , 0.57164331, 0.28514398, 0.        , 0.26258592,
        0.07645867, 0.10369715, 0.        , 0.31804633, 0.33670824]),
 'info': {'statusVal': 1,
  'iter': 220,
  'pobj': -0.21799326484017836,
  'dobj': -0.21799326484013704,
  'resPri': 8.335207321003254e-14,
  'resDual': 2.4501451719524393e-14,
  'relGap': 2.8780250536801384e-14,
  'resInfeas': nan,
  'resUnbdd': 4.587297688913491,
  'solveTime': 7.639045,
  'setupTime': 4.879494,
  'status': 'Solved'}}

In [11]:
np.random.seed(123)
(P, A, b, c) = gen_feasible(m, n, p_scale=0.)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
print(x)

probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 10, eps=1e-9, use_indirect=True)

probdata = dict(P=None, A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
scs.solve(probdata, cone, normalize=False, max_iters=N, acceleration_lookback = 10, eps=1e-9, use_indirect=True)

4999
[-0.09617189  0.63333043  0.29524907  0.20594927 -0.04627224 -0.09247665
 -0.05887478  0.38069075  0.31876822 -0.41737046]
----------------------------------------------------------------------------
	SCS v2.1.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 150, nnz in P = 0, cg_rate = 2.00
eps = 1.00e-09, alpha = 1.50, max_iters = 5000, normalize = 0
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 10, constraints m = 15
Cones:	linear vars: 15
Setup time: 6.03e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.31e+19  4.37e+18  9.01e-01 -6.13e+19 -3.21e+18  2.67e+19  6.65e-03 
    80| 8.16e-11  6.19e-11  5.70e-11 -2.18e-01 -2.18e-01  0.0

{'x': array([ 0.27820325,  0.66432651,  0.05226602, -0.60134236, -0.68370238,
        -1.26431774, -1.58822352,  1.68434295, -1.32041624, -1.31659194]),
 'y': array([0.        , 0.94962607, 0.269439  , 0.        , 0.        ,
        1.57242126, 0.        , 0.        , 1.20536578, 0.        ,
        0.        , 0.        , 1.42003202, 0.        , 0.        ]),
 's': array([0.54175045, 0.        , 0.        , 0.33533498, 0.32386352,
        0.        , 0.57279179, 0.25343078, 0.        , 0.26358338,
        0.03328514, 0.02755097, 0.        , 0.25585451, 0.30179785]),
 'info': {'statusVal': 1,
  'iter': 80,
  'pobj': -0.2179932647439538,
  'dobj': -0.2179932648258365,
  'resPri': 8.159880358469516e-11,
  'resDual': 6.190582647275608e-11,
  'relGap': 5.702191029141811e-11,
  'resInfeas': nan,
  'resUnbdd': 4.587297690677191,
  'solveTime': 6.8733,
  'setupTime': 4.54375,
  'status': 'Solved'}}